In [ ]:
import pandas as pd 
import numpy as np
import warnings
import sklearn
from sklearn.metrics import classification_report, f1_score
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
def stringify_labels(df):
  df_str_labels = []
  for z in range(0, df.shape[0]):
    str_labels = []
    if df['labels'][z][0] == 1:
      str_labels.append("neurotic")
    if df['labels'][z][1] == 1:
      str_labels.append("extravert")
    if df['labels'][z][2] == 1:
      str_labels.append("open")
    if df['labels'][z][3] == 1:
      str_labels.append("agreeable")
    if df['labels'][z][4] == 1:
      str_labels.append("conscious")

    df_str_labels.append(str_labels)

  len(df_str_labels)
  return df_str_labels

In [ ]:
def stringify_list_labels(ls):
  df_str_labels = []
  for z in range(0, len(ls)):
    str_labels = []
    if ls[z][0] == 1:
      str_labels.append("neurotic")
    if ls[z][1] == 1:
      str_labels.append("extravert")
    if ls[z][2] == 1:
      str_labels.append("open")
    if ls[z][3] == 1:
      str_labels.append("agreeable")
    if ls[z][4] == 1:
      str_labels.append("conscious")

    df_str_labels.append(str_labels)

  len(df_str_labels)
  return df_str_labels

The label list column is read as string. Need to convert it back to int 

In [ ]:
train = pd.read_csv('Joel\\datasets\\sds\\train_04.csv')
test = pd.read_csv('Joel\\datasets\\sds\\test.csv')
train["labels"] = [[int(x) for x in l if x.isnumeric()] for l in train["labels"]]
test["labels"] = [[int(x) for x in l if x.isnumeric()] for l in test["labels"]]

In [ ]:
train["str_labels"] = stringify_labels(train)
test["str_labels"] = stringify_labels(test)

In [ ]:
test.head(4)

,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
0,Durch Empfehlung. Über das Hide the Pain Harol...,0,0,0,0,0,"[0, 0, 0, 0, 0]"
1,Durch einen Rundgang. Weiß ich nicht mehr. Diä...,0,1,0,0,1,"[0, 1, 0, 0, 1]"
2,Ich bin im Josephs. Hallo. Ich habe Aufzeichnu...,1,1,1,1,0,"[1, 1, 1, 1, 0]"
3,Berufsschule. In der famiele. Heute morgen auf...,0,0,1,1,1,"[0, 0, 1, 1, 1]"


In [ ]:
def f1(predictions, outputs):
    mlb = MultiLabelBinarizer() # classes=["neuro_bin", "extra_bin", "off_bin", "ver_bin", "gew_bin"]
    #print("Outputs")
    #print(outputs)
    #print("Labels")
    #print(test["labels"])
    f1_scores= f1_score(
        mlb.fit_transform(list(test["labels"])),
        mlb.fit_transform(predictions),
        #average='weighted'
        average=None
    )
    #print(f1_scores)
    print(mlb.classes_)
    return f1_scores

def f1_manual(y_true, y_pred):
    mlb = MultiLabelBinarizer()
  
    f1_scores= f1_score(
        mlb.fit_transform(y_true),
        mlb.fit_transform(y_pred),
        #average='weighted'
        average=None
    )
    return f1_scores

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=5, use_cuda=True,
                                      args={'train_batch_size':2, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 

In [ ]:
# Train the model
#model.train_model(train, output_dir="Joel\\Transformers\\test2", acc=sklearn.metrics.accuracy_score, f1=sklearn.metrics.f1_score)
model.train_model(train, output_dir="Joel\\Transformers\\test2")

  0%|          | 0/56 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/28 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/28 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/28 [00:00<?, ?it/s]

(3, 1.2061945882936318)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test, f1=f1)
result
predictions = np.argmax(model_outputs,axis=1)
print(predictions)

  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

[0.0 1.0]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [ ]:
predictions, raw_outputs = model.predict(list(test['text']))

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(predictions)

[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]


In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel
model2 = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=5, use_cuda=True,
                                      args={'train_batch_size':4, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 6, 'max_seq_length': 512})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 

In [ ]:
# Train the model
model2.train_model(train, output_dir="Joel\\Transformers\\test2_1")

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 1 of 6:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 2 of 6:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 3 of 6:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 4 of 6:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 5 of 6:   0%|          | 0/16 [00:00<?, ?it/s]

(6, 0.6921318372090658)

In [ ]:
result, model_outputs, wrong_predictions = model2.eval_model(test)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(result)

{'LRAP': 0.7459201388888888, 'eval_loss': 0.6908990442752838}


In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel


model3 = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=5, use_cuda=True,
                                      args={'train_batch_size':4, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3})

# Train the model
model3.train_model(train, output_dir="Joel\\Transformers\\test3")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'c

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/16 [00:00<?, ?it/s]

(3, 0.6952793362239996)

In [ ]:
result, model_outputs, wrong_predictions = model3.eval_model(test)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(result)

{'LRAP': 0.714236111111111, 'eval_loss': 0.6932397782802582}


In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel


model4 = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=5, use_cuda=True,
                                      args={'train_batch_size': 2, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

# Train the model
model4.train_model(train, output_dir="Joel\\Transformers\\test4")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'c

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/32 [00:00<?, ?it/s]

(6, 0.6956941727548838)

In [ ]:
result, model_outputs, wrong_predictions = model4.eval_model(test)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'LRAP': 0.736892361111111, 'eval_loss': 0.6910165250301361}


In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel


model5 = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=5, use_cuda=True,
                                      args={'train_batch_size': 4, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

# Train the model
model5.train_model(train, output_dir="Joel\\Transformers\\test5")

In [ ]:
result, model_outputs, wrong_predictions = model5.eval_model(test)
print(result)

### More Transformer Models

In [ ]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
train = pd.read_csv('Joel\\datasets\\sds\\train.csv')
test = pd.read_csv('Joel\\datasets\\sds\\test.csv')

In [ ]:
train["labels"] = [[int(x) for x in l if x.isnumeric()] for l in train["labels"]]
test["labels"] = [[int(x) for x in l if x.isnumeric()] for l in test["labels"]]

In [ ]:
train.head()

,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
0,Durch eine Freundin von mir. Ich habe sehr gel...,1,1,1,1,0,"[1, 1, 1, 1, 0]"
1,Durch einen Schulausflug zur FAU in Nürnberg. ...,1,0,0,0,0,"[1, 0, 0, 0, 0]"
2,Ich bin hier einfach reingelaufen. An einem Ab...,0,1,1,1,0,"[0, 1, 1, 1, 0]"
3,"Email von einer Kollegin. Gestern, gemeinsam m...",0,1,1,0,0,"[0, 1, 1, 0, 0]"
4,Über Arbeitskollegen. Mir hat letztens jemand ...,1,0,0,1,1,"[1, 0, 0, 1, 1]"


In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=5, use_cuda=True,
                                      args={'train_batch_size': 2, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'c

In [ ]:
# Train the model
model.train_model(train, output_dir="Joel\\Transformers\\test2")

  0%|          | 0/56 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/28 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/28 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/28 [00:00<?, ?it/s]

(3, 1.2119671466449897)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
print(result)

  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'LRAP': 0.6984374999999999, 'eval_loss': 0.6997945308685303}


In [ ]:
print(wrong_predictions)

[{'guid': 0, 'text_a': 'Durch Empfehlung. Über das Hide the Pain Harold Meme mit meiner Mutter. Ich habe meinen Eltern beim Umzug geholfen. Wenn es halbwegs glatt läuft. Das globale Geschehen sowie verschiedene Gesellschaften und Kulturellen. Mit kleinen Dingen anfangen und öfters ein bisschen was dafür tun. Ich bin mit meinem Date an einem schönen Ort, vielleicht ein Strand und wir machen ein Picknick. Die Sonne scheint und das Meer wogt leise vor sich hin. Dannach gehen wir noch ein wenig spazieren und wir unterhalten uns über verschiedenste Themen. Bei dem Picknick gibt es natürlich gutes Essen, als erstes eine leichten Vorspeise wie Gemüse und etwas Käse oder ähnlichem und danach Brote mit verschiedenen Beilagen und Quark sowie Gemüse Cremes und später noch eine kleine Nachspeise wie einen Pudding oder Kuchen. Gegen Ende des Abends verabschieden wir uns und sprechen schon über das nächste Treffen und was wir unternehmen möchten.', 'text_b': None, 'label': [0, 0, 0, 0, 0]}, {'guid':

## Bert base multilingual

### Cased

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('bert', 'bert-base-multilingual-cased', num_labels=5, use_cuda=True,
                                      args={'train_batch_size':3, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassificatio

In [ ]:
# Train the model
model.train_model(train, output_dir="Joel\\Transformers\\test2")


  0%|          | 0/56 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/19 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/19 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/19 [00:00<?, ?it/s]

(3, 0.8259846369425455)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test, f1=f1)
print(result)

  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

[0.0 1.0]
{'LRAP': 0.7225694444444445, 'f1': array([1., 1.]), 'eval_loss': 0.6922067205111185}


In [ ]:
predictions, raw_outputs = model.predict(list(test['text']))

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
y_pred = stringify_list_labels(predictions)
y_true = stringify_list_labels(list(test['labels']))
print(y_pred)
print(y_true)

[['extravert', 'agreeable'], ['extravert', 'open', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'open', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'open', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'open', 'agreeable'], ['extravert', 'open', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'open', 'agreeable'], ['extravert', 'agreeable'], ['extravert', 'agreeable']]
[[], ['extravert', 'conscious'], ['neurotic', 'extravert', 'open', 'agreeable'], ['open', 'agreeable', 'conscious'], ['neurotic', 'open', 'agreeable', 'conscious'], ['neurotic', 'open'], [], ['neurotic', 'extravert', 'conscious'], ['extravert', 'conscious'], ['agreeable'], ['neu

In [ ]:
labels = list(test['labels'])
texts = list(test['text'])
for l in range(len(labels)):
  print("Text length: " + str(len(texts[l])))
  print("Pred: " + str(predictions[l]))
  print("Real: " + str(labels[l]))

Text length: 408
Pred: [0, 1, 0, 1, 0]
Real: [0, 1, 0, 1, 0]
Text length: 663
Pred: [0, 1, 0, 1, 1]
Real: [0, 0, 0, 0, 0]
Text length: 257
Pred: [0, 1, 0, 1, 0]
Real: [1, 1, 0, 0, 1]
Text length: 823
Pred: [0, 1, 0, 1, 0]
Real: [1, 0, 1, 0, 0]
Text length: 234
Pred: [0, 1, 0, 1, 1]
Real: [1, 1, 1, 1, 0]
Text length: 541
Pred: [0, 1, 0, 1, 1]
Real: [0, 1, 0, 1, 1]
Text length: 63
Pred: [0, 1, 0, 1, 0]
Real: [0, 0, 0, 1, 1]
Text length: 657
Pred: [0, 1, 0, 1, 0]
Real: [1, 0, 0, 1, 1]
Text length: 514
Pred: [0, 1, 0, 1, 1]
Real: [1, 1, 1, 1, 1]
Text length: 468
Pred: [0, 1, 0, 1, 1]
Real: [0, 0, 1, 1, 0]
Text length: 1067
Pred: [0, 1, 0, 1, 1]
Real: [1, 0, 0, 0, 0]
Text length: 167
Pred: [0, 1, 1, 1, 0]
Real: [1, 0, 1, 0, 0]
Text length: 251
Pred: [0, 1, 1, 1, 0]
Real: [0, 1, 0, 1, 1]
Text length: 240
Pred: [0, 1, 0, 1, 0]
Real: [0, 1, 0, 1, 1]
Text length: 136
Pred: [0, 1, 1, 1, 0]
Real: [0, 0, 1, 1, 1]
Text length: 868
Pred: [0, 1, 0, 1, 0]
Real: [1, 0, 1, 0, 1]


### Uncased

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('bert', 'bert-base-multilingual-uncased', num_labels=5, use_cuda=True,
                                      args={'train_batch_size':4, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassificat

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
print(result)

  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'LRAP': 0.7247685185185185, 'eval_loss': 0.6944092512130737}


## Bert Base German Cased

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('bert', 'bert-base-german-cased', num_labels=5, use_cuda=True,
                                      args={'train_batch_size':4, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
print(result)

  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'LRAP': 0.7104745370370371, 'eval_loss': 0.6974496444066366}


## distilbert-base-multilingual-cased

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('distilbert', 'distilbert-base-multilingual-cased', num_labels=5, use_cuda=True,
                                      args={'train_batch_size':3, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-4, 'num_train_epochs': 3})

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForMultiLabelSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultiLabelSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initial

In [ ]:
# Train the model
model.train_model(train, output_dir="Joel\\Transformers\\test2")

  0%|          | 0/56 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/19 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/19 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/19 [00:00<?, ?it/s]

(3, 0.8127696911493937)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
print(result)

  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'LRAP': 0.7225694444444445, 'eval_loss': 0.7192927797635397}


In [ ]:
predictions, raw_outputs = model.predict(list(test['text']))
print(predictions)
print(list(test["labels"]))

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[[1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 1, 1, 0], [1, 1, 0, 1, 0], [0, 1, 1, 1, 0]]
[[0, 0, 0, 0, 0], [0, 1, 0, 0, 1], [1, 1, 1, 1, 0], [0, 0, 1, 1, 1], [1, 0, 1, 1, 1], [1, 0, 1, 0, 0], [0, 0, 0, 0, 0], [1, 1, 0, 0, 1], [0, 1, 0, 0, 1], [0, 0, 0, 1, 0], [1, 0, 0, 1, 1], [1, 0, 0, 1, 1], [0, 1, 0, 1, 1], [0, 1, 0, 0, 1], [1, 0, 1, 1, 0], [0, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 0, 0, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 0, 1, 1], [1, 0, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0]]


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
print(result)

  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'LRAP': 0.769386574074074, 'eval_loss': 0.6900294820467631}


## XLMRoberta

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel

model = MultiLabelClassificationModel('xlmroberta', 'xlm-roberta-base', num_labels=5, use_cuda=True,
                                      args={'train_batch_size':3, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 5, 'max_seq_length': 512})

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMultiLabelSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
# Train the model
model.train_model(train, output_dir="Joel\\Transformers\\test2")

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/22 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/22 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/22 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/22 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/22 [00:00<?, ?it/s]

(5, 0.9576439216732979)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'LRAP': 0.7145833333333333, 'eval_loss': 0.6913168728351593}


In [ ]:
predictions, raw_outputs = model.predict(list(test['text']))

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(raw_outputs)

[[0.3972168  0.47241211 0.48095703 0.5078125  0.59130859]
 [0.43164062 0.50341797 0.47143555 0.50439453 0.57958984]
 [0.40673828 0.47973633 0.47949219 0.5078125  0.58935547]
 [0.44482422 0.51611328 0.46508789 0.50048828 0.56884766]
 [0.40356445 0.47900391 0.48095703 0.50585938 0.58837891]
 [0.39990234 0.48071289 0.47827148 0.5078125  0.58740234]
 [0.40698242 0.48144531 0.47802734 0.5078125  0.58789062]
 [0.41210938 0.48071289 0.47875977 0.5        0.59082031]
 [0.40454102 0.47875977 0.48071289 0.50390625 0.58740234]
 [0.40844727 0.48095703 0.47998047 0.50488281 0.58935547]
 [0.40380859 0.48168945 0.47729492 0.50439453 0.58789062]
 [0.40966797 0.48681641 0.47753906 0.50585938 0.58349609]
 [0.41772461 0.49023438 0.47827148 0.50439453 0.58154297]
 [0.40380859 0.47851562 0.47875977 0.50634766 0.58789062]
 [0.41577148 0.48779297 0.47729492 0.50732422 0.58349609]
 [0.42993164 0.50537109 0.46923828 0.5        0.578125  ]
 [0.40771484 0.48266602 0.47753906 0.50830078 0.58642578]
 [0.40698242 0

In [ ]:
print(predictions)
print(list(test["labels"]))

[[0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0], [0, 0, 1, 1, 0]]
[[0, 1, 0, 1, 0], [0, 0, 0, 0, 0], [1, 1, 0, 0, 1], [1, 0, 1, 0, 0], [1, 1, 1, 1, 0], [0, 1, 0, 1, 1], [0, 0, 0, 1, 1], [1, 0, 0, 1, 1], [1, 1, 1, 1, 1], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 1, 0, 0], [0, 1, 0, 1, 1], [0, 1, 0, 1, 1], [0, 0, 1, 1, 1], [1, 0, 1, 0, 1]]
